## 1. Cài đặt thư viện

In [ ]:
!pip install -q llama-index llama-index-llms-openai llama-index-embeddings-openai llama-index-vector-stores-faiss
!pip install -q faiss-cpu beautifulsoup4 requests tqdm scikit-learn

## 2. Import libraries và setup

In [ ]:
import os
import json
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
from typing import List, Dict

# LlamaIndex
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# Setup OpenAI API key từ Colab Secrets
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

print("✅ Setup completed!")

## 3. Cấu hình LlamaIndex và GRPO Format với Enhanced Requirements

In [ ]:
# Cấu hình LLM và Embedding
llm = OpenAI(model="gpt-4o-mini", temperature=0.7)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Set global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 50

# Define GRPO reasoning format markers
reasoning_start = "<start_working_out>"
reasoning_end = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

# Enhanced requirements with word ranges
MIN_REASONING_WORDS = 200
MAX_REASONING_WORDS = 300
MIN_SOLUTION_WORDS = 200
MAX_SOLUTION_WORDS = 400

print("✅ LlamaIndex configured with GPT-4o-mini")
print(f"✅ GRPO format markers configured:")
print(f"   Reasoning: {reasoning_start} ... {reasoning_end} ({MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} words)")
print(f"   Solution: {solution_start} ... {solution_end} ({MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} words)")

## 4. Crawl văn bản pháp luật từ ThuvienPhapluat.vn

In [ ]:
def crawl_legal_document_thuvienphapluat(url: str) -> str:
    """
    Crawl nội dung văn bản pháp luật từ ThuvienPhapluat.vn
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm nội dung chính - ThuvienPhapluat thường dùng các selectors này
        content_text = ""

        # Cách 1: Tìm div có class hoặc id chứa nội dung
        selectors = [
            {'class': 'content1'},
            {'class': 'fulltext'},
            {'class': 'noidung'},
            {'class': 'content'},
            {'id': 'content'},
            {'class': 'content-text'},
            {'class': 'lawtext'},
            {'class': 'law-content'}
        ]

        for selector in selectors:
            content_div = soup.find('div', selector)
            if content_div:
                content_text = content_div.get_text(separator='\n', strip=True)
                if len(content_text) > 1000:  # Nội dung đủ dài
                    break

        # Cách 2: Tìm trong các thẻ article hoặc main
        if len(content_text) < 1000:
            article = soup.find('article')
            if article:
                content_text = article.get_text(separator='\n', strip=True)
        
        # Cách 3: Tìm các div có id hoặc class chứa 'text', 'content'
        if len(content_text) < 1000:
            for div in soup.find_all('div'):
                if div.get('class') and any('text' in cls.lower() or 'content' in cls.lower() for cls in div.get('class')):
                    text = div.get_text(separator='\n', strip=True)
                    if len(text) > len(content_text):
                        content_text = text

        # Cách 4: Nếu không tìm thấy, tìm trong body và loại bỏ script, style
        if len(content_text) < 1000:
            # Xóa các thẻ không cần thiết
            for tag in soup(['script', 'style', 'nav', 'header', 'footer', 'aside', 'menu']):
                tag.decompose()

            # Lấy text từ body
            body = soup.find('body')
            if body:
                content_text = body.get_text(separator='\n', strip=True)

        # Làm sạch text
        content_text = re.sub(r'\n\s*\n', '\n\n', content_text)
        lines = content_text.split('\n')
        cleaned_lines = [line for line in lines if len(line.strip()) > 10]
        content_text = '\n'.join(cleaned_lines)

        return content_text

    except Exception as e:
        print(f"❌ Error crawling {url}: {e}")
        return ""

# Crawl từ 2 URLs được yêu cầu
document_urls = [
    "https://thuvienphapluat.vn/van-ban/Giao-thong-Van-tai/Luat-trat-tu-an-toan-giao-thong-duong-bo-2024-so-36-2024-QH15-444251.aspx",
    "https://thuvienphapluat.vn/van-ban/Giao-thong-Van-tai/Luat-Duong-bo-2024-588811.aspx"
]

all_legal_text = ""

for i, url in enumerate(document_urls, 1):
    print(f"📥 Crawling document {i} from: {url}")
    legal_text = crawl_legal_document_thuvienphapluat(url)
    
    if legal_text:
        all_legal_text += f"\n\n=== LUẬT {i} ===\n\n" + legal_text
        print(f"✅ Crawled {len(legal_text):,} characters from document {i}")
    else:
        print(f"❌ Failed to crawl document {i}")

print(f"\n✅ Total crawled: {len(all_legal_text):,} characters")
print(f"📄 Number of lines: {len(all_legal_text.splitlines())}")
print(f"\nPreview (first 1000 chars):\n{all_legal_text[:1000]}...")

## 5. Chunking và tạo Documents

In [ ]:
# Tạo Document từ text crawled
document = Document(text=all_legal_text)

# Chunking với SentenceSplitter (tăng chunk size để phù hợp với yêu cầu enhanced)
splitter = SentenceSplitter(
    chunk_size=800,  # Tăng chunk size để có nhiều context hơn
    chunk_overlap=100,
    separator="\n"
)

nodes = splitter.get_nodes_from_documents([document])

print(f"✅ Created {len(nodes)} chunks")
print(f"\nSample chunk:\n{nodes[0].text[:300]}...")

## 6. Tạo FAISS Vector Store và merge similar chunks

In [ ]:
# Tạo FAISS index
dimension = 1536  # text-embedding-3-small dimension
faiss_index = faiss.IndexFlatL2(dimension)

# Tạo vector store
vector_store = FaissVectorStore(faiss_index=faiss_index)

# Tạo index từ nodes
print("🔄 Creating vector index...")
index = VectorStoreIndex(
    nodes=nodes,
    vector_store=vector_store,
    show_progress=True
)

print(f"✅ Vector index created with {len(nodes)} nodes")

In [ ]:
def merge_similar_chunks(nodes: List, similarity_threshold: float = 0.70) -> List[Dict]:
    """
    Merge các chunks có cosine similarity cao để tạo contexts phong phú hơn
    Giảm threshold để tạo contexts lớn hơn phù hợp với yêu cầu 300+ words
    """
    print(f"🔄 Merging similar chunks (threshold={similarity_threshold})...")
    print(f"📊 Input: {len(nodes)} chunks")

    # Get embeddings cho tất cả nodes
    embeddings = []
    for node in tqdm(nodes, desc="Getting embeddings"):
        emb = embed_model.get_text_embedding(node.text)
        embeddings.append(emb)

    embeddings = np.array(embeddings)

    # Tính cosine similarity matrix
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_matrix = cosine_similarity(embeddings)

    # Merge chunks
    merged_contexts = []
    used_indices = set()

    for i in range(len(nodes)):
        if i in used_indices:
            continue

        # Tìm các chunks similar
        similar_indices = np.where(similarity_matrix[i] > similarity_threshold)[0]
        similar_indices = [idx for idx in similar_indices if idx not in used_indices]

        # Merge text
        merged_text = "\n\n".join([nodes[idx].text for idx in similar_indices])

        # Chỉ keep contexts đủ lớn để tạo ra reasoning và solution dài
        if len(merged_text.split()) >= 150:  # Minimum context size
            merged_contexts.append({
                'context': merged_text,
                'num_chunks': len(similar_indices),
                'chunk_indices': similar_indices
            })

        used_indices.update(similar_indices)

    print(f"✅ Merged {len(nodes)} chunks into {len(merged_contexts)} contexts")
    print(f"📈 Avg chunks per context: {len(nodes)/len(merged_contexts):.1f}")
    return merged_contexts

# Merge chunks với threshold thấp hơn để tạo contexts lớn hơn
merged_contexts = merge_similar_chunks(nodes, similarity_threshold=0.70)
print(f"\n📊 Number of merged contexts: {len(merged_contexts)}")

## 7. Generate Questions từ Contexts

In [ ]:
def generate_questions_from_context(context: str, num_questions: int = 8) -> List[str]:
    """
    Generate câu hỏi phức tạp hơn từ context để phù hợp với enhanced requirements
    """
    prompt = f"""Bạn là chuyên gia pháp luật giao thông Việt Nam. Dựa trên đoạn văn bản pháp luật dưới đây, hãy tạo {num_questions} câu hỏi hay, phức tạp và đa dạng mà người dùng có thể hỏi.

Văn bản pháp luật:
{context}

Yêu cầu:
- Câu hỏi phải phức tạp, yêu cầu phân tích sâu và giải thích chi tiết
- Đa dạng về loại: hỏi định nghĩa chi tiết, so sánh, phân tích tình huống, thủ tục phức tạp, quyền và nghĩa vụ
- Câu hỏi cần có thể tạo ra câu trả lời dài ít nhất 300 từ
- Ưu tiên các câu hỏi về tình huống thực tế, case study, hoặc yêu cầu giải thích logic pháp lý
- Câu hỏi phải có thể trả lời được từ context trên
- Trả về ONLY danh sách các câu hỏi, mỗi câu một dòng, không đánh số

Câu hỏi:"""

    try:
        response = llm.complete(prompt)
        questions_text = response.text.strip()

        # Parse questions
        questions = [q.strip() for q in questions_text.split('\n') if q.strip()]
        # Loại bỏ numbering nếu có
        questions = [re.sub(r'^\d+\.\s*', '', q) for q in questions]

        return questions[:num_questions]
    except Exception as e:
        print(f"❌ Error generating questions: {e}")
        return []

# Test với 1 context
test_context = merged_contexts[0]['context']
test_questions = generate_questions_from_context(test_context, num_questions=8)

print("📝 Sample generated questions:")
for i, q in enumerate(test_questions, 1):
    print(f"{i}. {q}")

## 8. Generate Enhanced Structured Answers cho GRPO Format

In [ ]:
def generate_enhanced_structured_answer(question: str, context: str) -> str:
    """
    Generate câu trả lời có cấu trúc GRPO format với yêu cầu enhanced:
    - Reasoning 200-300 từ
    - Solution 200-400 từ
    """
    prompt = f"""Bạn là trợ lý AI chuyên sâu về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích chi tiết trong phần {reasoning_start} {reasoning_end} ({MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} từ)
2. Đưa ra câu trả lời chi tiết trong phần {solution_start}{solution_end} ({MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} từ)

Văn bản pháp luật:
{context}

Câu hỏi: {question}

Yêu cầu cho phần suy nghĩ (working_out) - {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} từ:
- Phân tích câu hỏi: người dùng muốn biết gì cụ thể?
- Xác định và liệt kê các điều luật, khoản, điểm liên quan
- Phân tích từng quy định một cách chi tiết
- Xem xét các trường hợp đặc biệt, ngoại lệ
- So sánh với các quy định khác nếu có
- Đánh giá tác động và ý nghĩa của quy định
- Giải thích logic và nguyên tắc pháp lý đằng sau

Yêu cầu cho phần solution - {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} từ:
- Trả lời trực tiếp và toàn diện câu hỏi
- Trích dẫn chính xác các điều luật, khoản, điểm
- Giải thích rõ ràng các khái niệm pháp lý
- Đưa ra ví dụ cụ thể nếu phù hợp
- Hướng dẫn thực hiện hoặc tuân thủ
- Lưu ý các điểm quan trọng cần chú ý
- Tham khảo thêm nếu cần thiết

Format bắt buộc:
{reasoning_start}
[Phần phân tích và suy nghĩ chi tiết - {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} từ]
{reasoning_end}

{solution_start}[Câu trả lời chi tiết và toàn diện - {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} từ]{solution_end}"""

    try:
        response = llm.complete(prompt)
        answer = response.text.strip()

        # Validate format và độ dài
        if reasoning_start in answer and reasoning_end in answer and solution_start in answer and solution_end in answer:
            # Extract và check word count
            reasoning_section = extract_reasoning_section(answer)
            solution_section = extract_solution_section(answer)
            
            reasoning_words = len(reasoning_section.split())
            solution_words = len(solution_section.split())
            
            # If either section doesn't meet word count requirements, regenerate with emphasis
            reasoning_valid = MIN_REASONING_WORDS <= reasoning_words <= MAX_REASONING_WORDS
            solution_valid = MIN_SOLUTION_WORDS <= solution_words <= MAX_SOLUTION_WORDS
            
            if not reasoning_valid or not solution_valid:
                print(f"⚠️ Regenerating - Reasoning: {reasoning_words} (need {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS}), Solution: {solution_words} (need {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS})")
                
                # Try again with more emphasis
                enhanced_prompt = prompt + f"\n\nQUAN TRỌNG: Reasoning phải có {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} từ, Solution phải có {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} từ. Hãy đảm bảo độ dài chính xác!"
                response = llm.complete(enhanced_prompt)
                answer = response.text.strip()
            
            return answer
        else:
            # Fallback: create structure if not properly formatted
            simple_answer = response.text.strip()
            structured = f"""{reasoning_start}
Để trả lời câu hỏi này một cách toàn diện, tôi cần phân tích chi tiết văn bản pháp luật được cung cấp và xem xét tất cả các khía cạnh liên quan đến: {question}

Trước tiên, tôi sẽ xác định các điều khoản pháp luật áp dụng trực tiếp cho vấn đề này. Việc phân tích này đòi hỏi sự hiểu biết sâu sắc về hệ thống pháp luật giao thông Việt Nam và cách các quy định được áp dụng trong thực tế.

Tiếp theo, tôi cần xem xét bối cảnh lịch sử và mục đích của các quy định này, để hiểu rõ ý định của nhà làm luật và cách thức áp dụng phù hợp. Điều này bao gồm việc phân tích các nguyên tắc cơ bản của luật giao thông, các quyền và nghĩa vụ của các chủ thể tham gia giao thông, và mối quan hệ giữa các quy định khác nhau trong hệ thống pháp luật.

Cuối cùng, tôi sẽ đánh giá các tác động thực tiễn và đưa ra câu trả lời dựa trên phân tích pháp lý chính xác và toàn diện.
{reasoning_end}

{solution_start}{simple_answer}{solution_end}"""
            return structured

    except Exception as e:
        print(f"❌ Error generating enhanced structured answer: {e}")
        return ""

def extract_reasoning_section(answer):
    """Extract text between reasoning markers"""
    try:
        start_idx = answer.find(reasoning_start) + len(reasoning_start)
        end_idx = answer.find(reasoning_end)
        return answer[start_idx:end_idx].strip()
    except:
        return ""

def extract_solution_section(answer):
    """Extract text between solution markers"""
    try:
        start_idx = answer.find(solution_start) + len(solution_start)
        end_idx = answer.find(solution_end)
        return answer[start_idx:end_idx].strip()
    except:
        return ""

# Test với 1 question
test_enhanced_answer = generate_enhanced_structured_answer(test_questions[0], test_context)

print("💬 Sample Enhanced Structured Q&A:")
print(f"Q: {test_questions[0]}")
print(f"A: {test_enhanced_answer[:800]}...")

# Check word counts
reasoning_section = extract_reasoning_section(test_enhanced_answer)
solution_section = extract_solution_section(test_enhanced_answer)
print(f"\n📊 Word counts - Reasoning: {len(reasoning_section.split())}, Solution: {len(solution_section.split())}")
print("\n" + "="*80)

## 9. Generate toàn bộ Enhanced GRPO dataset (Target: 1000+ samples)

In [ ]:
def generate_enhanced_grpo_dataset(
    contexts: List[Dict],
    target_samples: int = 1000,
    questions_per_context: int = 20
) -> List[Dict]:
    """
    Generate enhanced synthetic dataset với GRPO structured format
    Yêu cầu: reasoning 200-300 từ, solution 200-400 từ
    """
    dataset = []
    failed_generations = 0

    # Tính số contexts cần dùng
    num_contexts_needed = min(len(contexts), (target_samples // questions_per_context) + 1)

    print(f"🚀 Generating {target_samples} Enhanced GRPO Q&A pairs from {num_contexts_needed} contexts...")
    print(f"📏 Requirements: Reasoning {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} words, Solution {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} words")

    for ctx_idx, ctx_data in enumerate(tqdm(contexts[:num_contexts_needed], desc="Processing contexts")):
        context = ctx_data['context']

        # Skip contexts quá ngắn
        if len(context.split()) < 100:
            continue

        # Generate questions
        questions = generate_questions_from_context(context, num_questions=questions_per_context)

        # Generate enhanced structured answers cho mỗi question
        for q_idx, question in enumerate(questions):
            if len(dataset) >= target_samples:
                break

            print(f"\r🔄 Context {ctx_idx+1}/{num_contexts_needed}, Q {q_idx+1}/{len(questions)}, Total: {len(dataset)}/{target_samples}", end="")
            
            structured_answer = generate_enhanced_structured_answer(question, context)

            if structured_answer and reasoning_start in structured_answer:
                # Validate word counts
                reasoning_section = extract_reasoning_section(structured_answer)
                solution_section = extract_solution_section(structured_answer)
                
                reasoning_words = len(reasoning_section.split())
                solution_words = len(solution_section.split())
                
                # Only add if meets word count requirements
                reasoning_valid = MIN_REASONING_WORDS <= reasoning_words <= MAX_REASONING_WORDS
                solution_valid = MIN_SOLUTION_WORDS <= solution_words <= MAX_SOLUTION_WORDS
                
                if reasoning_valid and solution_valid:
                    dataset.append({
                        'question': question,
                        'answer': structured_answer,
                        'format': 'enhanced_grpo_structured',
                        'reasoning_words': reasoning_words,
                        'solution_words': solution_words,
                        'context_length': len(context.split())
                    })
                else:
                    failed_generations += 1
                    print(f"\n⚠️ Skipped: Reasoning {reasoning_words} (need {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS}) or Solution {solution_words} (need {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS})")
            else:
                failed_generations += 1

        if len(dataset) >= target_samples:
            break

    print(f"\n\n✅ Generated {len(dataset)} Enhanced GRPO Q&A pairs")
    print(f"❌ Failed generations: {failed_generations}")
    return dataset

# Generate enhanced dataset
enhanced_grpo_data = generate_enhanced_grpo_dataset(
    merged_contexts,
    target_samples=1000,
    questions_per_context=20
)

## 10. Lưu Enhanced GRPO dataset dưới dạng JSONL

In [ ]:
# Lưu ra file JSONL với tên phân biệt
output_path = 'enhanced_synthetic_legal_qa_grpo_format.jsonl'

with open(output_path, 'w', encoding='utf-8') as f:
    for item in enhanced_grpo_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ Saved {len(enhanced_grpo_data)} Enhanced GRPO Q&A pairs to {output_path}")

# Download file (nếu chạy trên Colab)
from google.colab import files
files.download(output_path)

## 11. Thống kê & Preview Enhanced GRPO dataset

In [ ]:
import pandas as pd

# Tạo DataFrame để phân tích
df = pd.DataFrame(enhanced_grpo_data)

# Thống kê độ dài
df['question_len'] = df['question'].apply(lambda x: len(x.split()))
df['answer_len'] = df['answer'].apply(lambda x: len(x.split()))

print("📊 ENHANCED GRPO DATASET STATISTICS:")
print("=" * 60)
print(f"Total samples: {len(df):,}")
print(f"All samples meet enhanced requirements: ✅")
print(f"\nQuestion length (words):")
print(df['question_len'].describe())
print(f"\nAnswer length (words):")
print(df['answer_len'].describe())
print(f"\nReasoning section length (words):")
print(df['reasoning_words'].describe())
print(f"\nSolution section length (words):")
print(df['solution_words'].describe())
print(f"\nContext length (words):")
print(df['context_length'].describe())

# Quality checks
print(f"\n📏 QUALITY CHECKS:")
reasoning_in_range = ((df['reasoning_words'] >= MIN_REASONING_WORDS) & (df['reasoning_words'] <= MAX_REASONING_WORDS))
solution_in_range = ((df['solution_words'] >= MIN_SOLUTION_WORDS) & (df['solution_words'] <= MAX_SOLUTION_WORDS))
print(f"Reasoning sections {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} words: {reasoning_in_range.sum()}/{len(df)} ({reasoning_in_range.mean()*100:.1f}%)")
print(f"Solution sections {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} words: {solution_in_range.sum()}/{len(df)} ({solution_in_range.mean()*100:.1f}%)")

# Preview samples
print("\n" + "=" * 60)
print("SAMPLE ENHANCED GRPO Q&A PAIRS:")
print("=" * 60)
for i in range(min(2, len(df))):
    print(f"\n[Sample {i+1}]")
    print(f"Q: {df.iloc[i]['question']}")
    print(f"A: {df.iloc[i]['answer'][:600]}...")
    print(f"📊 Stats: R={df.iloc[i]['reasoning_words']}w, S={df.iloc[i]['solution_words']}w, Total={df.iloc[i]['answer_len']}w")
    print("-" * 60)

## 12. Enhanced Validation & Quality Check

In [ ]:
# Kiểm tra chất lượng format Enhanced GRPO
print("🔍 ENHANCED GRPO FORMAT QUALITY CHECKS:")
print("=" * 60)

# Check duplicates
duplicate_questions = df[df.duplicated(subset=['question'], keep=False)]
print(f"Duplicate questions: {len(duplicate_questions)}")

# Check format structure
has_reasoning = df['answer'].apply(lambda x: reasoning_start in x and reasoning_end in x)
has_solution = df['answer'].apply(lambda x: solution_start in x and solution_end in x)
proper_format = has_reasoning & has_solution

print(f"Has reasoning format: {has_reasoning.sum()}/{len(df)} ({has_reasoning.mean()*100:.1f}%)")
print(f"Has solution format: {has_solution.sum()}/{len(df)} ({has_solution.mean()*100:.1f}%)")
print(f"Proper format: {proper_format.sum()}/{len(df)} ({proper_format.mean()*100:.1f}%)")

# Check enhanced requirements compliance
meets_reasoning_req = (df['reasoning_words'] >= MIN_REASONING_WORDS) & (df['reasoning_words'] <= MAX_REASONING_WORDS)
meets_solution_req = (df['solution_words'] >= MIN_SOLUTION_WORDS) & (df['solution_words'] <= MAX_SOLUTION_WORDS)
meets_both_req = meets_reasoning_req & meets_solution_req

print(f"\n📏 ENHANCED REQUIREMENTS COMPLIANCE:")
print(f"Reasoning {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} words: {meets_reasoning_req.sum()}/{len(df)} ({meets_reasoning_req.mean()*100:.1f}%)")
print(f"Solution {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} words: {meets_solution_req.sum()}/{len(df)} ({meets_solution_req.mean()*100:.1f}%)")
print(f"Meets both requirements: {meets_both_req.sum()}/{len(df)} ({meets_both_req.mean()*100:.1f}%)")

# Distribution analysis
print(f"\n📈 LENGTH DISTRIBUTION:")
print(f"Reasoning words - Min: {df['reasoning_words'].min()}, Max: {df['reasoning_words'].max()}, Avg: {df['reasoning_words'].mean():.1f}")
print(f"Solution words - Min: {df['solution_words'].min()}, Max: {df['solution_words'].max()}, Avg: {df['solution_words'].mean():.1f}")
print(f"Total answer words - Min: {df['answer_len'].min()}, Max: {df['answer_len'].max()}, Avg: {df['answer_len'].mean():.1f}")

print("\n✅ Enhanced GRPO format quality check completed!")

# Sample detailed validation
if len(df) > 0:
    print("\n" + "=" * 60)
    print("SAMPLE DETAILED VALIDATION:")
    sample_answer = df.iloc[0]['answer']
    reasoning_text = extract_reasoning_section(sample_answer)
    solution_text = extract_solution_section(sample_answer)

    reasoning_valid = MIN_REASONING_WORDS <= len(reasoning_text.split()) <= MAX_REASONING_WORDS
    solution_valid = MIN_SOLUTION_WORDS <= len(solution_text.split()) <= MAX_SOLUTION_WORDS

    print(f"\nExtracted Reasoning ({len(reasoning_text.split())} words):\n{reasoning_text[:400]}...")
    print(f"\nExtracted Solution ({len(solution_text.split())} words):\n{solution_text[:400]}...")
    print(f"\nFormat valid: {bool(reasoning_text and solution_text)}")
    print(f"Requirements met: R({MIN_REASONING_WORDS}-{MAX_REASONING_WORDS}){'✅' if reasoning_valid else '❌'}, S({MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS}){'✅' if solution_valid else '❌'}")

## 13. Export cho Enhanced SFT Training format

In [ ]:
# Tạo enhanced format training cho SFT với system prompt
enhanced_system_prompt = f"""Bạn là một trợ lý AI chuyên sâu về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích chi tiết câu hỏi trong phần {reasoning_start} {reasoning_end} ({MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} từ)
2. Đưa ra câu trả lời toàn diện trong phần {solution_start}{solution_end} ({MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} từ)

Phần suy nghĩ cần bao gồm: phân tích câu hỏi, xác định điều luật liên quan, giải thích logic pháp lý, và đánh giá các khía cạnh khác nhau.
Phần giải đáp cần: trả lời trực tiếp, trích dẫn chính xác pháp luật, giải thích rõ ràng, và đưa ra hướng dẫn cụ thể.

Câu trả lời cần dựa trên quy định pháp luật hiện hành và phải chi tiết, toàn diện, dễ hiểu."""

# Convert sang format training
enhanced_training_data = []
for item in enhanced_grpo_data:
    reasoning_valid = MIN_REASONING_WORDS <= item['reasoning_words'] <= MAX_REASONING_WORDS
    solution_valid = MIN_SOLUTION_WORDS <= item['solution_words'] <= MAX_SOLUTION_WORDS
    
    if (item['answer'] and 
        reasoning_start in item['answer'] and
        reasoning_valid and solution_valid):
        
        training_item = {
            "messages": [
                {"role": "system", "content": enhanced_system_prompt},
                {"role": "user", "content": item['question']},
                {"role": "assistant", "content": item['answer']}
            ],
            "metadata": {
                "reasoning_words": item['reasoning_words'],
                "solution_words": item['solution_words'],
                "format": "enhanced_grpo"
            }
        }
        enhanced_training_data.append(training_item)

# Lưu enhanced training format
enhanced_training_output_path = 'enhanced_synthetic_legal_qa_grpo_training.jsonl'

with open(enhanced_training_output_path, 'w', encoding='utf-8') as f:
    for item in enhanced_training_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ Saved {len(enhanced_training_data)} enhanced training examples to {enhanced_training_output_path}")
print(f"📈 Ready for Enhanced SFT training with GRPO format!")

# Download training file
files.download(enhanced_training_output_path)

print("\n🎯 ENHANCED SUMMARY:")
print(f"- Crawled from 2 legal documents: Luật trật tự ATGT đường bộ 2024 & Luật Đường bộ 2024")
print(f"- Generated {len(enhanced_grpo_data)} enhanced synthetic samples")
print(f"- {len(enhanced_training_data)} samples meet enhanced requirements")
print(f"- Reasoning sections: {MIN_REASONING_WORDS}-{MAX_REASONING_WORDS} words (avg: {df['reasoning_words'].mean():.0f})")
print(f"- Solution sections: {MIN_SOLUTION_WORDS}-{MAX_SOLUTION_WORDS} words (avg: {df['solution_words'].mean():.0f})")
print(f"- Format includes detailed {reasoning_start}...{reasoning_end} and {solution_start}...{solution_end}")
print(f"- Ready for Enhanced SFT training with comprehensive reasoning and solutions")